### main.ipynb
- load data
- train model

In [1]:
import gendata
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing, TopKPooling
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.utils import add_self_loops, degree

import random
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# start time log
time_start = time.time()

# load data -> Dataset
data_path_good = r"D:\Desktop\hybrid-SVD\CWE23_good\dot"
data_path_bad = r"D:\Desktop\hybrid-SVD\CWE23_bad\dot"

Dataset = gendata.gendata(data_path_good)
Dataset = Dataset + gendata.gendata(data_path_bad)

random.shuffle(Dataset)

lenDataset = len(Dataset)
len_Trainset = int(0.7 * lenDataset)
len_Validateset = int(0.2 * lenDataset)
len_Testset = lenDataset - len_Validateset - len_Trainset
Trainset = Dataset[:len_Trainset]
Validset = Dataset[len_Trainset: len_Validateset]
Testset = Dataset[len_Trainset + len_Validateset:]
print(f"len of dataset:{lenDataset}, \
        len of train:{len_Trainset}, \
        len of validate:{len_Validateset}, \
        len of test:{len_Testset}")

# dataset -> loader
train_loader = DataLoader(Trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(Testset, batch_size=32, shuffle=True)
good = 0
bad = 0
for test in Testset:
    if test["y"] == 1:
        good += 1
    else:
        bad += 1
print(f"good in testset:{good};;bad in testset:{bad}")

# fin load ; print time span
time_fin_load = time.time()
print("data load time: ", time_fin_load-time_start)

del === 0
del === 0
del === 0
del === 0
del === 0
del === 19
del === 0
del === 0
del === 19
del === 0
del === 19
del === 0
del === 0
del === 19
del === 0
del === 19
del === 0
del === 0
del === 19
del === 0
del === 19
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 16
del === 0
del === 0
del === 16
del === 0
del === 16
del === 0
del === 0
del === 16
del === 0
del === 16
del === 0
del === 0
del === 16
del === 0
del === 16
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del === 0
del ==

In [3]:
# GCN layer
# cite: https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, **kwargs):
        super().__init__(aggr='add', **kwargs)  # "Add" aggregation (Step 5).
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.lin = Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x[N, in_channels]
        # edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        row, col = edge_index

        deg = degree(col, x.size(0), dtype=x.dtype)

        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        return  self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]
        # Step 6: Return new node embeddings.
        return aggr_out

In [4]:
# Whole NN model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(256, 256)
        # self.conv1 = GraphConv(128, 128)
        self.pool1 = TopKPooling(256, ratio=0.8)
        self.conv2 = GCNConv(256, 256)
        # self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(256, ratio=0.8)
        self.conv3 = GCNConv(256, 256)
        # self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(256, ratio=0.8)
        self.conv4 = GCNConv(256, 256)
        self.pool4 = TopKPooling(256, ratio=0.8)
        self.conv5 = GCNConv(256, 256)
        self.pool5 = TopKPooling(256, ratio=0.8)

        self.convAtt1 = torch.nn.Conv1d(in_channels=512, out_channels=64, kernel_size=1, stride=2)
        self.poolAtt1 = torch.nn.MaxPool1d(kernel_size=1, stride=2)
        self.convAtt2 = torch.nn.Conv1d(64, 16, kernel_size=1, stride=2)
        self.poolAtt2 = torch.nn.MaxPool1d(kernel_size=1, stride=2)
        self.convAtt3 = torch.nn.Conv1d(16, 2, kernel_size=1, stride=2)
        self.poolAtt3 = torch.nn.MaxPool1d(kernel_size=1, stride=2)

        self.convAtt4 = torch.nn.Conv1d(2, 16, kernel_size=1, stride=2)
        self.poolAtt4 = torch.nn.MaxPool1d(kernel_size=1, stride=2)
        self.convAtt5 = torch.nn.Conv1d(16, 64, kernel_size=1, stride=2)
        self.poolAtt5 = torch.nn.MaxPool1d(kernel_size=1, stride=2)
        self.convAtt6 = torch.nn.Conv1d(64, 512, kernel_size=1, stride=2)
        self.poolAtt6 = torch.nn.MaxPool1d(kernel_size=1, stride=2)

        self.lin1 = torch.nn.Linear(512, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 2)

        self.readout = Seq(Linear(128, 64),
                           ReLU(),
                           Linear(64, 2))

    def forward(self, x, edge_index, batch):
        # x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = F.relu(self.conv4(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool4(x, edge_index, None, batch)
        x4 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = F.relu(self.conv5(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool5(x, edge_index, None, batch)
        x5 = torch.cat([global_max_pool(x, batch), global_mean_pool(x, batch)], dim=1)

        x = x1 + x2 + x3 + x4 + x5

        x = 1/5 * x

        sx = x

        x = x.unsqueeze(dim=2)
        # attention层
        x = F.relu(self.convAtt1(x))
        x = self.poolAtt1(x)
        x = F.relu(self.convAtt2(x))
        x = self.poolAtt2(x)
        x = F.relu(self.convAtt3(x))
        x = self.poolAtt3(x)

        x = F.relu(self.convAtt4(x))
        x = self.poolAtt4(x)
        x = F.relu(self.convAtt5(x))
        x = self.poolAtt5(x)
        x = F.relu(self.convAtt6(x))
        x = self.poolAtt6(x)

        x = x.squeeze()

        x = (x + 1) * sx

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.log_softmax(self.lin3(x), dim=-1)

        return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
def train(epoch):
    model.train()
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        x, edge_index, batch = data.x, data.edge_index, data.batch
        output = model(x, edge_index, batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(Trainset)

In [7]:
def test(t_loader):
    model.eval()
    correct = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    pred_good = 0
    pred_bad = 0
    for data in t_loader:
        data = data.to(device)
        x, edge_index, batch = data.x, data.edge_index, data.batch
        preds = model(x, edge_index, batch).max(dim=1)[1]
        correct += preds.eq(data.y).sum().item()
        for i in range(len(preds)):
            if preds[i] == 0 and data.y[i] == 0:
                tp += 1
                pred_bad += 1
            elif preds[i] == 0 and data.y[i] == 1:
                fp += 1
                pred_bad += 1
            elif preds[i] == 1 and data.y[i] == 1:
                tn += 1
                pred_good += 1
            elif preds[i] == 1 and data.y[i] == 0:
                fn += 1
                pred_good += 1
    acc = (correct/len(t_loader.dataset))
    return round(acc, 6), tp, fp, tn, fn, pred_good, pred_bad

In [8]:
# train & test
# time log
time_start = time.time()

train_loss_a = np.zeros(200)
test_acc_a = np.zeros(200)
for epoch in range(0, 200):
    loss = train(epoch)
    train_acc, tp, fp, tn, fn, pred_good, pred_bad = test(train_loader)
    test_acc, tp, fp, tn, fn, pred_good, pred_bad = test(test_loader)
    train_loss_a[epoch] = loss
    test_acc_a[epoch] = test_acc
    print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f},Test Acc: {:.5f}, TP: {:02d}, FP: {:02d}, TN: {:02d}, FN: {:02d}, Pred_good: {:04d}, Pred_bad: {:04d}'.
          format(epoch, loss, train_acc, test_acc, tp, fp, tn, fn, pred_good, pred_bad))

# fin train & test;;print time span
time_fin_train = time.time()
print("train & test time span : ", time_fin_train - time_start)

Epoch: 000, Loss: 0.64985, Train Acc: 0.89881,Test Acc: 0.91146, TP: 89, FP: 14, TN: 86, FN: 03, Pred_good: 0089, Pred_bad: 0103
Epoch: 001, Loss: 0.30730, Train Acc: 0.88616,Test Acc: 0.89062, TP: 71, FP: 00, TN: 100, FN: 21, Pred_good: 0121, Pred_bad: 0071
Epoch: 002, Loss: 0.23579, Train Acc: 0.90625,Test Acc: 0.91146, TP: 89, FP: 14, TN: 86, FN: 03, Pred_good: 0089, Pred_bad: 0103
Epoch: 003, Loss: 0.21590, Train Acc: 0.90625,Test Acc: 0.91146, TP: 75, FP: 00, TN: 100, FN: 17, Pred_good: 0117, Pred_bad: 0075
Epoch: 004, Loss: 0.21242, Train Acc: 0.90774,Test Acc: 0.91667, TP: 90, FP: 14, TN: 86, FN: 02, Pred_good: 0088, Pred_bad: 0104
Epoch: 005, Loss: 0.18037, Train Acc: 0.90625,Test Acc: 0.92188, TP: 83, FP: 06, TN: 94, FN: 09, Pred_good: 0103, Pred_bad: 0089
Epoch: 006, Loss: 0.15478, Train Acc: 0.90774,Test Acc: 0.91146, TP: 75, FP: 00, TN: 100, FN: 17, Pred_good: 0117, Pred_bad: 0075
Epoch: 007, Loss: 0.18161, Train Acc: 0.90327,Test Acc: 0.90104, TP: 84, FP: 11, TN: 89, FN: 0

In [9]:
print(device)

cuda
